In [1]:
%reload_ext autoreload
%autoreload 2

# How to submit to the leaderboard


In this tutorial, we will build a graph-convolutional neural network for the PBE bandgap task.
We will use the [Crystal Graph Convolutional Neural Network](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.120.145301) implemented in the [Deepchem library](https://deepchem.io/).

We choose this one, as it will need some more involved training loop and customization that you might also need in a real-world scenario.


To install Deepchem, follow the installation instructions on [the Deepchem landing page](https://deepchem.io/). You'll need to run something along the following lines (and also install PyTorch and dgl):

> conda install -c conda-forge rdkit deepchem==2.6.1
>
> pip install tensorflow-gpu~=2.4


## Imports


In [2]:
import json
import os
import uuid
from typing import Iterable, List, Tuple, Union

import deepchem as dc
import numpy as np
from deepchem.data import Dataset
from deepchem.data.data_loader import InMemoryLoader
from deepchem.feat import MaterialStructureFeaturizer
from deepchem.feat.graph_data import GraphData
from deepchem.models.torch_models.cgcnn import CGCNNModel
from deepchem.molnet.load_function.molnet_loader import TransformerGenerator, _MolnetLoader
from deepchem.utils.data_utils import download_url, get_data_dir
from deepchem.utils.typing import PymatgenStructure
from loguru import logger
from pymatgen.analysis.local_env import CrystalNN, CutOffDictNN, JmolNN
from pymatgen.core import Structure
from pymatgen.core.structure import Structure

from mofdscribe.bench import PBEBandGapIDBench

ATOM_INIT_JSON_URL = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/atom_init.json"
VESTA_NN = CutOffDictNN.from_preset("vesta_2019")

## Plumbing to make Deepchem work with custom datasets


In [3]:
model = CGCNNModel(
    mode="regression", in_edge_dim=16
)  # we use default settings for demonstration purposes. We change the edge dim as we us a simpler featurizer.


We use a simpler featurizer, however, you can also use the default `CGCNNFeaturizer`


In [4]:
class CrystalBondFeaturizer(MaterialStructureFeaturizer):
    """
    Calculate structure graph features for crystals.

    Based on the implementation in Crystal Graph Convolutional
    Neural Networks (CGCNN). The method constructs a crystal graph
    representation including atom features and bond features (neighbor
    distances). Neighbors are determined using bond heuristics.
    Optionally, a Gaussian filter is applied to neighbor distances.
    All units are in Angstrom.
    This featurizer requires the optional dependency pymatgen. It may
    be useful when 3D coordinates are available and when using graph
    network models and crystal graph convolutional networks.
    See [1]_ for more details.
    References
    ----------
    .. [1] T. Xie and J. C. Grossman, "Crystal graph convolutional
       neural networks for an accurate and interpretable prediction
       of material properties", Phys. Rev. Lett. 120, 2018,
       https://arxiv.org/abs/1710.10324
    Examples
    --------
    >>> import pymatgen as mg
    >>> featurizer = CrystalBondFeaturizer()
    >>> lattice = mg.core.Lattice.cubic(4.2)
    >>> structure = mg.core.Structure(lattice, ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
    >>> features = featurizer.featurize([structure])
    >>> feature = features[0]
    >>> print(type(feature))
    <class 'deepchem.feat.graph_data.GraphData'>
    Note
    ----
    This class requires Pymatgen to be installed.
    """

    def __init__(self, heuristic: str = "vesta", step: float = 0.2, radius: float = 3.0):
        """Initialize CrystalBondFeaturizer.

        Parameters
        ----------
        heuristic : str
            The heuristic to use for determining neighbors.
        radius : float
            Radius of sphere for finding neighbors of atoms in unit cell. This is the radius
            of the Gaussian filter. Default is 3.0.
        step : float
            Step size for Gaussian filter. This value is used when building edge features.
            If None, use only the bond length. Default is 0.2.

        Raises
        ------
        ValueError
            If `heuristic` is not one of the following: ["vesta", "jmol", "crystal"].
        """
        heuristic = heuristic.lower()
        if heuristic == "vesta":
            self.nn = VESTA_NN
        elif heuristic == "jmol":
            self.nn = JmolNN()
        elif heuristic == "crystal":
            self.nn = CrystalNN()
        else:
            raise ValueError("Unknown heuristic: {}".format(heuristic))
        self.step = step
        self.radius = radius
        # load atom_init.json
        data_dir = get_data_dir()
        download_url(ATOM_INIT_JSON_URL, data_dir)
        atom_init_json_path = os.path.join(data_dir, "atom_init.json")
        with open(atom_init_json_path, "r") as f:
            atom_init_json = json.load(f)

        self.atom_features = {
            int(key): np.array(value, dtype=np.float32) for key, value in atom_init_json.items()
        }
        self.valid_atom_number = set(self.atom_features.keys())

    def _featurize(self, datapoint: PymatgenStructure, **kwargs) -> GraphData:
        """Calculate crystal graph features from pymatgen structure.

        Parameters
        ----------
        datapoint: pymatgen.core.Structure
                A periodic crystal composed of a lattice and a sequence of atomic
                sites with 3D coordinates and elements.

        Returns
        -------
        graph: GraphData
                A crystal graph with CGCNN style features.
        """
        if type(datapoint) is not Structure:
            logger.warning(
                f"CrystalBondFeaturizer requires pymatgen.core.Structure, got {type(datapoint)}"
            )
            raise ValueError(
                f"CrystalBondFeaturizer requires pymatgen.core.Structure, got {type(datapoint)}"
            )
        node_features = self._get_node_features(datapoint)
        edge_index, edge_features = self._get_edge_features_and_index(datapoint)
        graph = GraphData(node_features, edge_index, edge_features)
        return graph

    def _get_node_features(self, struct: PymatgenStructure) -> np.ndarray:
        """Get the node feature from `atom_init.json`.

        The `atom_init.json` was collected
        from `data/sample-regression/atom_init.json` in the CGCNN repository.

        Parameters
        ----------
        struct: pymatgen.core.Structure
            A periodic crystal composed of a lattice and a sequence of atomic
            sites with 3D coordinates and elements.

        Returns
        -------
        node_features: np.ndarray
            A numpy array of shape `(num_nodes, 92)`.
        """
        node_features = []
        for site in struct:
            # check whether the atom feature exists or not
            if site.specie.number not in self.valid_atom_number:
                raise RuntimeError("site.specie.number not in self.valid_atom_number")
            node_features.append(self.atom_features[site.specie.number])
        return np.vstack(node_features).astype(float)

    def _get_edge_features_and_index(
        self, struct: PymatgenStructure
    ) -> Tuple[np.ndarray, np.ndarray]:
        """Calculate the edge feature and edge index from pymatgen structure.

        Parameters
        ----------
        struct: pymatgen.core.Structure
            A periodic crystal composed of a lattice and a sequence of atomic
            sites with 3D coordinates and elements.
        Returns
        -------
        edge_idx: np.ndarray, dtype int
            A numpy array of shape with `(2, num_edges)`.
        edge_features: np.ndarray
            A numpy array of shape with `(num_edges, filter_length)`. The `filter_length` is
            (self.radius / self.step) + 1. The edge features were built by applying gaussian
            filter to the distance between nodes.
        """
        neighbors_ = self.nn.get_all_nn_info(struct)

        neighbors = []
        for n in neighbors_:
            sites = [s["site"] for s in n]
            n = sorted(sites, key=lambda x: x[1])
            neighbors.append(n)

        # construct bi-directed graph
        src_idx, dest_idx = [], []
        edge_distances = []
        for node_idx, neighbor in enumerate(neighbors):
            src_idx.extend([node_idx] * len(neighbor))
            dest_idx.extend([site[2] for site in neighbor])
            edge_distances.extend([site[1] for site in neighbor])

        edge_idx = np.array([src_idx, dest_idx], dtype=int)

        if self.step is None:
            edge_features = np.array(edge_distances)
        else:
            edge_features = self._gaussian_filter(np.array(edge_distances, dtype=float))
        return edge_idx, edge_features

    def _gaussian_filter(self, distances: np.ndarray) -> np.ndarray:
        """Apply Gaussian filter to an array of interatomic distances.

        Parameters
        ----------
        distances : np.ndarray
                A numpy array of the shape `(num_edges, )`.
        Returns
        -------
        expanded_distances: np.ndarray
                Expanded distance tensor after Gaussian filtering.
                The shape is `(num_edges, filter_length)`. The `filter_length` is
                (self.radius / self.step) + 1.
        """
        filt = np.arange(0, self.radius + self.step, self.step)

        # Increase dimension of distance tensor and apply filter
        expanded_distances = np.exp(-((distances[..., np.newaxis] - filt) ** 2) / self.step ** 2)

        return expanded_distances


We now implement a loader that will take the structures and labels. Some notes:

- we convert to lists as the splitters by default return generators.


In [5]:
class StructureDataLoader(_MolnetLoader):
    """StructureDataLoader loader.

    This data loader assumes that there is a folder with subfolder `cifs`.
    The `cifs` subfolders contains all the cif files to be loaded.

    Labels are loaded from a json-serialized file in the folder which
    name can be specfied with `label_file_name`.

    Note that there will be errors if the structures do not _exactly_ match
    the entries in the json file.

    Parameters
    ----------
    featurizer : Union[dc.feat.Featurizer, str]
            the featurizer to use for processing the data.  Alternatively you can pass
            one of the names from dc.molnet.featurizers as a shortcut.
    splitter : Union[dc.splits.Splitter, str], optional
            the splitter to use for splitting the data into training, validation, and
            test sets.  Alternatively you can pass one of the names from
            dc.molnet.splitters as a shortcut.  If this is None, all the data
            will be included in a single dataset.
    transformer_generators : List[Union[TransformerGenerator, str]]
            the Transformers to apply to the data.  Each one is specified by a
            TransformerGenerator or, as a shortcut, one of the names from
            dc.molnet.transformers.
    tasks : List[str]
            the names of the tasks in the dataset
    data_dir : Optional[str]
            a directory to save the raw data in
    save_dir : Optional[str]
            a directory to save the dataset in
    label_file_name : Optional[str]
            the name of the json file containing the labels. Defaults to `qmof.json`.
    identifier_column : Optional[str]
            the name of the column that contains the identifier of the structure.
            Defaults to `qmof_id`.
    """

    def __init__(
        self,
        structures: Iterable[Structure],
        labels: Iterable[float],
        idx: Iterable[int],
        splitter: Union[str, dc.splits.Splitter] = None,  # "random",
        transformer_generators: List[Union[str, TransformerGenerator]] = ["normalization"],
        tasks: List[str] = ["qmof"],
        data_dir: str = None,
        save_dir: str = None,
        number_files: int = np.infty,
        shard_size: int = 8,
        n_workers: int = 1,
    ):
        # we return IStructures, however, the featurizer wants structures.
        self.structures = [[Structure.from_sites(s.sites)] for s in structures]

        self.labels = (
            np.array(list(labels)).reshape(-1, 1)
            if labels is not None
            else [np.nan] * len(self.structures)
        )

        self.idx = np.array(list(idx))

        self.number_files = number_files

        self.shard_size = shard_size
        self.n_workers = n_workers

        super().__init__(
            featurizer=CrystalBondFeaturizer(),
            splitter=splitter,
            transformer_generators=transformer_generators,
            tasks=tasks,
            data_dir=data_dir,
            save_dir=save_dir,
        )

    def create_dataset(self) -> Dataset:
        """Utilitary function to create the dataset."""
        loader = InMemoryLoader(
            tasks=self.tasks,
            featurizer=self.featurizer,
        )

        return loader.create_dataset(
            list(zip(self.structures, self.labels)),
            shard_size=self.shard_size,
            #     n_workers=self.n_workers,
        )


Let's check our plumbing works.


We initialize a bench class to access the dataset.


In [7]:
bench = PBEBandGapIDBench(model, "test", debug=True)


2022-08-16 08:37:08.980 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:256 - Dropped 0 duplicate basenames. New length 15042
2022-08-16 08:37:09.059 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:262 - Dropped 136 duplicate graphs. New length 14906
2022-08-16 08:37:19.795 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:256 - Dropped 0 duplicate basenames. New length 15042
2022-08-16 08:37:19.865 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:262 - Dropped 136 duplicate graphs. New length 14906
2022-08-16 08:37:20.094 | DEBUG    | mofdscribe.splitters.splitters:__init__:116 - Splitter settings | shuffle True, random state None, sample frac 0.01, q (0, 0.25, 0.5, 0.75, 1)


In [8]:
# get some random indices
indices = np.random.choice(np.arange(len(bench._ds)), 20)

structures = list(bench._ds.get_structures(indices))
labels = bench._ds._df[bench._targets].iloc[indices].values


In [9]:
loader = StructureDataLoader(structures, labels, indices, data_dir="test-data", save_dir="test")


In [10]:
ds = loader.load_dataset("qmof-test", False)


In [11]:
ds


(['qmof'],
 (<DiskDataset X.shape: (20,), y.shape: (20, 1), w.shape: (20, 1), ids: [0 1 2 ... 17 18 19], task_names: ['qmof']>,),
 [<deepchem.trans.transformers.NormalizationTransformer at 0x2a192ce20>])

In [12]:
model.fit(ds[1][0])


/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0.16975947618484497

## Implementing a model class for `Bench`


Now, that we now that the plumbing works, we only have to wrap it into a class.


In [13]:
class MyCGCNNModel:
    def __init__(self, model):
        self.model = model
        self.transformers = None
        self._fitted = False

    def _create_ds(self, structures, labels, indices):
        loader = StructureDataLoader(structures, labels, indices, data_dir=None, save_dir=None)
        # Deepchem likes to reload things, even if the content changed.
        # To improve performance, you might want to pre-compute the graphs and then load them from
        # disk in this step.
        ds = loader.load_dataset(str(uuid.uuid1()), False)

        return ds[1], ds[2]

    def fit(self, idx, structures, y):
        structures = list(structures)
        datasets, transformers = self._create_ds(structures, y, idx)
        self.transformers = transformers
        self.model.fit(datasets[0])
        self._fitted = True

    def predict(self, idx, structures):
        if not self._fitted:
            raise Exception("Model not fitted")
        structures = list(structures)
        datasets, transformers = self._create_ds(structures, None, idx)
        datasets = transformers[0].transform(datasets[0])
        logger.debug("Datset len {}".format(len(datasets)))
        pred = self.model.predict(datasets, transformers=self.transformers).flatten()
        return pred


In [14]:
mymodel = MyCGCNNModel(model)


In [15]:
mymodel.fit(indices, structures, labels)


/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [16]:
mymodel.predict(indices, structures)


2022-08-16 08:38:56.538 | DEBUG    | __main__:predict:29 - Datset len 20


array([1.76906876, 1.64655768, 2.16257102, 1.42804826, 2.13805037,
       2.05404935, 2.06023867, 2.45996961, 2.28364652, 1.69595277,
       1.64224082, 1.57646053, 1.92568648, 2.06816744, 1.65583512,
       1.57138615, 1.62099518, 2.08708934, 2.04884018, 2.40747122])

It seems to do something reasonable. So, let's try running it in the `Bench` (with `debug=True`, which runs the benchmark with only 1% of the data).


In [30]:
bench = PBEBandGapIDBench(
    MyCGCNNModel(CGCNNModel(mode="regression", in_edge_dim=16)),
    "deepchem-cgcnn-vesta-edges",
    features="n/a",
    model_type="CGCNN",
    debug=True,
)


2022-08-02 13:57:32.673 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:120 - Dropped 0 duplicate basenames. New length 15844
2022-08-02 13:57:32.688 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:126 - Dropped 153 duplicate graphs. New length 15691
2022-08-02 13:57:33.504 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:120 - Dropped 0 duplicate basenames. New length 15844
2022-08-02 13:57:33.517 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:126 - Dropped 153 duplicate graphs. New length 15691
2022-08-02 13:57:33.567 | DEBUG    | mofdscribe.splitters.splitters:__init__:106 - Splitter settings | shuffle True, random state None, sample frac 0.01, q (0, 0.25, 0.5, 0.75, 1)


In [31]:
report = bench.bench()


2022-08-02 13:57:34.628 | DEBUG    | mofdscribe.bench.mofbench:_score:190 - K-fold round 0, 128 train points, 28 test points
/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warn

In [34]:
report


BenchResult(start_time=datetime.datetime(2022, 8, 2, 11, 57, 33, 635334, tzinfo=datetime.timezone.utc), end_time=datetime.datetime(2022, 8, 2, 12, 2, 34, 792392, tzinfo=datetime.timezone.utc), metrics=RegressionMetricCollection(regression_metrics=[RegressionMetrics(mean_squared_error=1.0472316550218441, mean_absolute_error=0.8838351196121169, r2_score=0.05848829701812652, max_error=2.080018412271101, mean_absolute_percentage_error=0.7899650207873089, top_5_in_top_5=0, top_10_in_top_10=1, top_50_in_top_50=1, top_100_in_top_100=1, top_500_in_top_500=1), RegressionMetrics(mean_squared_error=1.0824931011805925, mean_absolute_error=0.8779317214402516, r2_score=0.41164418914671375, max_error=2.0744940787669943, mean_absolute_percentage_error=1.3785416950726006, top_5_in_top_5=1, top_10_in_top_10=1, top_50_in_top_50=1, top_100_in_top_100=1, top_500_in_top_500=1), RegressionMetrics(mean_squared_error=1.044226503252123, mean_absolute_error=0.8482118945912059, r2_score=0.3278387849116352, max_er

Now, let's see how we can create a report file for the leaderboard.


In [33]:
report.save_json("test-deepchem")


Now, that we seem to get some reasonable results. For this, we'll run the full pipeline in a Python script as this is more handy for use on computing clusters (with multiprocessing).


In [20]:
bench = PBEBandGapIDBench(
    MyCGCNNModel(CGCNNModel(mode="regression", num_conv=4, in_edge_dim=16)),
    "deepchem-cgcnn-vesta-edges",
    features="n/a",
    model_type="CGCNN",
    debug=False,
)


2022-08-16 21:23:48.411 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:256 - Dropped 0 duplicate basenames. New length 15042
2022-08-16 21:23:48.508 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:262 - Dropped 136 duplicate graphs. New length 14906
2022-08-16 21:24:03.109 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:256 - Dropped 0 duplicate basenames. New length 15042
2022-08-16 21:24:03.197 | DEBUG    | mofdscribe.datasets.qmof_dataset:__init__:262 - Dropped 136 duplicate graphs. New length 14906
2022-08-16 21:24:03.463 | DEBUG    | mofdscribe.splitters.splitters:__init__:116 - Splitter settings | shuffle True, random state None, sample frac 1.0, q (0, 0.25, 0.5, 0.75, 1)


In [21]:
report = bench.bench()


2022-08-16 21:29:31.813 | DEBUG    | mofdscribe.bench.mofbench:_score:322 - K-fold round 0, 1012 train points, 261 test points
/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/kevinmaikjablonka/miniconda3/envs/mofdscribe/lib/python3.8/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  wa

In [23]:
report.save_json('/Users/kevinmaikjablonka/git/kjappelbaum/mofdscribe/bench_results/pbe_bandgap_id')

In [24]:
report.save_rst('/Users/kevinmaikjablonka/git/kjappelbaum/mofdscribe/bench_results/pbe_bandgap_id')